In [1]:
import os
print("当前工作目录：", os.getcwd())
import torch
import torchvision
from torchvision import datasets, transforms
# from CL_task_agnostic.loss_surface import     

当前工作目录： d:\Githubdesktop\github\thesis_code\thesis_code\CL_task_agnostic


In [ ]:
from tensorboard.backend.event_processing import event_accumulator
import numpy as np
import pandas as pd


# 1. 指定事件文件路径
event_file = r"D:\Githubdesktop\github\thesis_code\thesis_code\log_CL_task_agnostic\none\criterion=GDL_lr=0.001_lr_reduce_batch=2_seed=43\events.out.tfevents.1747542899.pc3054.3998252.0"
print("路径：", event_file)
print("存在？", os.path.exists(event_file))
print("是文件？", os.path.isfile(event_file))
if os.path.isfile(event_file):
    print("大小 (bytes)：", os.path.getsize(event_file))


# 2. 加载：告诉它 scalars 全都读出来（0 表示不限制数量）
ea = event_accumulator.EventAccumulator(
    event_file,
    size_guidance={event_accumulator.SCALARS: 0}
)
ea.Reload()

# 3. 查看有哪些 scalar tags
print("可用 tags:", ea.Tags()["scalars"])

# 4. 取出 train_loss 对应的所有条目
#    每个 entry 是一个 namedtuple： (wall_time, step, value)
entries = ea.Scalars("total train loss")

# 5. 转成列表 / NumPy /DataFrame
steps      = [e.step for e in entries]
wall_times = [e.wall_time for e in entries]
values     = [e.value for e in entries]

# 作为 Python 列表
loss_list_training = values
print('loss_list len', len(loss_list_training))

# 或转 NumPy 数组
loss_arr = np.array(values)

# Loss_surface detection
count_updates=0
loss_window_length = 8
loss_window_mean_threshold = 0.25
loss_window_variance_threshold = 1e-4
jump_ratio_threshold = 0.5
loss_window=[]
loss_window_means=[]
loss_window_variances=[]
new_peak_detected = True
num_batches = len(loss_list_training)
curr_batch_loss = None
prev_batch_loss = None
for i in range(num_batches):
    loss = loss_list_training[i]
    loss_window.append(np.mean(loss))
    if len(loss_window)>loss_window_length: del loss_window[0]
    loss_window_mean=np.mean(loss_window)
    loss_window_variance=np.var(loss_window)
    # Save previous loss window mean and variance

    ##
    prev_batch_loss = curr_batch_loss
    curr_batch_loss = loss
    if prev_batch_loss is not None and curr_batch_loss is not None:
        jump_ratio = (curr_batch_loss - prev_batch_loss) / prev_batch_loss 

    # print(f'batch: {i+1}loss window mean: {loss_window_mean}, loss window variance: {loss_window_variance} jump ratio: {jump_ratio}')
    # if not new_peak_detected and loss_window_mean > last_loss_window_mean+np.sqrt(last_loss_window_variance) :
    if not new_peak_detected and loss_window_mean > prev_loss_window_mean+np.sqrt(prev_loss_window_variance) and jump_ratio > jump_ratio_threshold:
        new_peak_detected=True 
        count_updates+=1
        print(f'new peak detected at {i+1} step')
    
    if loss_window_mean < loss_window_mean_threshold and loss_window_variance < loss_window_variance_threshold and new_peak_detected:
        print(f'Detect loss plateau at {i+1} step for {count_updates} times: last loss window mean: {loss_window_mean}, last loss window variance: {loss_window_variance}')
        last_loss_window_mean=loss_window_mean
        last_loss_window_variance=loss_window_variance
        new_peak_detected=False
    
    prev_loss_window_mean     = loss_window_mean
    prev_loss_window_variance = loss_window_variance
    
print(f'count_updates: {count_updates}')



# last loss window mean: 0.18679414689540863, last loss window variance: 8.96894832769668e-05

In [4]:
def detect_loss_surface(loss_list_training, loss_window_length=8, loss_window_mean_threshold=0.25, loss_window_variance_threshold=1e-4, jump_ratio_threshold=0.5):
    loss_window = []
    jump_ratio_threshold = 0.5
    count_updates=0
    new_peak_detected = True
    prev_batch_loss = None
    curr_batch_loss = None
    for i in range(len(loss_list_training)):
        loss = loss_list_training[i]
        loss_window.append(np.mean(loss))
        if len(loss_window)>loss_window_length: del loss_window[0]
        loss_window_mean=np.mean(loss_window)
        loss_window_variance=np.var(loss_window)
        # Save previous loss window mean and variance

        ##
        prev_batch_loss = curr_batch_loss
        curr_batch_loss = loss
        if prev_batch_loss is not None and curr_batch_loss is not None:
            jump_ratio = (curr_batch_loss - prev_batch_loss) / prev_batch_loss 

        # print(f'batch: {i+1}loss window mean: {loss_window_mean}, loss window variance: {loss_window_variance} jump ratio: {jump_ratio}')
        # if not new_peak_detected and loss_window_mean > last_loss_window_mean+np.sqrt(last_loss_window_variance) :
        if not new_peak_detected :
            if loss_window_mean > prev_loss_window_mean+np.sqrt(prev_loss_window_variance) and jump_ratio > jump_ratio_threshold:
            # if loss_window_mean > prev_loss_window_mean+np.sqrt(prev_loss_window_variance) :
                new_peak_detected=True 
                count_updates+=1
                print(f'new peak detected at {i+1} step')
        
        if loss_window_mean < loss_window_mean_threshold and loss_window_variance < loss_window_variance_threshold and new_peak_detected:
            print(f'Detect loss plateau at {i+1} step for {count_updates} times: last loss window mean: {loss_window_mean}, last loss window variance: {loss_window_variance}')
            last_loss_window_mean=loss_window_mean
            last_loss_window_variance=loss_window_variance
            new_peak_detected=False
        
        prev_loss_window_mean     = loss_window_mean
        prev_loss_window_variance = loss_window_variance
        
    print(f'count_updates: {count_updates}')

In [5]:
# VSCode.Cell id="new_detect_gdl" language="python"
import os
import numpy as np
from tensorboard.backend.event_processing import event_accumulator

root_dir = r"D:\Githubdesktop\github\thesis_code\thesis_code\log_CL_task_agnostic\ewc"

# 1) 递归查找所有以 criterion=GDL 开头的子目录
gdl_dirs = []
for root, dirs, _ in os.walk(root_dir):
    for d in dirs:
        if d.startswith("criterion=GDL"):
            gdl_dirs.append(os.path.join(root, d))
print(f"找到 {len(gdl_dirs)} 个 GDL 目录：")
# for d in gdl_dirs:
    # print("  ", d)

# 2) 在这些目录下收集所有 events.out.tfevents 文件
event_files = []
for d in gdl_dirs:
    for fn in os.listdir(d):
        if fn.startswith("events.out.tfevents"):
            event_files.append(os.path.join(d, fn))
print(f"共找到 {len(event_files)} 个 event 文件：")
for ef in event_files:
    print("  ", ef)

# 3) 验证加载并打印可用的 scalar tags
i = 0
loss_window_length = 7
loss_window_mean_threshold = 0.25
loss_window_variance_threshold = 1e-4
jump_ratio_threshold = 0.5
for ef in event_files:
    try:
        print(gdl_dirs[i])
        i += 1  
        ea = event_accumulator.EventAccumulator(
            ef,
            size_guidance={event_accumulator.SCALARS: 0}
        )
        ea.Reload()
        entries = ea.Scalars("train loss")

        # 5. 转成列表 / NumPy /DataFrame
        steps      = [e.step for e in entries]
        wall_times = [e.wall_time for e in entries]
        values     = [e.value for e in entries]

        # 作为 Python 列表
        loss_list_training = values
        print('loss_list len', len(loss_list_training))

        # 或转 NumPy 数组
        loss_arr = np.array(values)
        detect_loss_surface(loss_list_training, loss_window_length=loss_window_length, loss_window_mean_threshold=loss_window_mean_threshold, loss_window_variance_threshold=loss_window_variance_threshold, jump_ratio_threshold=jump_ratio_threshold)
    except Exception as e:
        print(f"[FAIL] {os.path.basename(ef)} → {e}")

找到 4 个 GDL 目录：
共找到 4 个 event 文件：
   D:\Githubdesktop\github\thesis_code\thesis_code\log_CL_task_agnostic\ewc\criterion=GDL_lr=0.001_lr_reduce_batch=3_seed=42_ewc_lambda=0.4_ewc_alpha=0.9\events.out.tfevents.1747572525.pc3054.3204022.0
   D:\Githubdesktop\github\thesis_code\thesis_code\log_CL_task_agnostic\ewc\criterion=GDL_lr=0.001_lr_reduce_batch=3_seed=42_ewc_lambda=1.0_ewc_alpha=0.9\events.out.tfevents.1747572490.pc3054.3196219.0
   D:\Githubdesktop\github\thesis_code\thesis_code\log_CL_task_agnostic\ewc\criterion=GDL_lr=0.001_lr_reduce_batch=3_seed=43_ewc_lambda=1.0_ewc_alpha=0.9\events.out.tfevents.1747572490.pc3054.3196278.0
   D:\Githubdesktop\github\thesis_code\thesis_code\log_CL_task_agnostic\ewc\criterion=GDL_lr=0.001_lr_reduce_batch=3_seed=44_ewc_lambda=1.0_ewc_alpha=0.9\events.out.tfevents.1747572513.pc3054.3196426.0
D:\Githubdesktop\github\thesis_code\thesis_code\log_CL_task_agnostic\ewc\criterion=GDL_lr=0.001_lr_reduce_batch=3_seed=42_ewc_lambda=0.4_ewc_alpha=0.9
loss_lis